In [ ]:
!pip install -q pgmpy networkx matplotlib plotly daft

In [ ]:
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.inference import VariableElimination

In [ ]:
from pgmpy.factors.discrete.CPD import TabularCPD

def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

In [ ]:
#Q01
statenames = {
'TrainStrike': ['True', 'False'],
'PeterLate': ['True', 'False'],
'JohannaLate': ['True', 'False'],
'PetersAlarmFails': ['True', 'False']}

model = BayesianNetwork([('PetersAlarmFails', 'PeterLate'),
                         ('TrainStrike', 'PeterLate'),
                         ('TrainStrike', 'JohannaLate')])

model.to_daft().render()

# Define the probability distribution for 'PetersAlarmFails' (P(PetersAlarmFails))
cpd_petersalarmfails = TabularCPD(variable='PetersAlarmFails', variable_card=2,
                          state_names=statenames,
                          values=[[0.1], [0.9]])
# Define the probability distribution for 'TrainStrike' (P(TrainStrike))
cpd_trainstrike = TabularCPD(variable='TrainStrike', variable_card=2,
                          state_names=statenames,
                          values=[[0.05], [0.95]])

# Define the conditional probability distribution for 'Test' (P(Test | Infected))
cpd_peterlate = TabularCPD(variable='PeterLate', variable_card=2,
                      state_names=statenames,
                      evidence=['TrainStrike','PetersAlarmFails'], evidence_card=[2,2],
                      values = [[0.7, 0.3, 0.4, 0.01],
                                [0.3, 0.7, 0.6, 0.99]])

cpd_johannalate = TabularCPD(variable='JohannaLate', variable_card=2,
                      state_names=statenames,
                      evidence=['TrainStrike',], evidence_card=[2],
                      values = [[0.5, 0.02],
                                [0.5, 0.98]])


model.add_cpds(cpd_petersalarmfails, cpd_trainstrike, cpd_peterlate, cpd_johannalate)
# print the conditional probability distributions
for cpd in model.get_cpds():
  print_full(cpd)



In [ ]:
#Q02
inference = VariableElimination(model)

print(inference.query(variables=['PeterLate']))
print(inference.query(variables=['JohannaLate']))


In [ ]:
#Q03
inference = VariableElimination(model)

evidence={'PetersAlarmFails':'True'}
result = inference.query(variables=['JohannaLate'], evidence=evidence)
print(evidence)
print(result)

In [ ]:
#Q04
inference = VariableElimination(model)

evidence={'PeterLate':'True'}
result = inference.query(variables=['JohannaLate'], evidence=evidence)
print(evidence)
print(result)

In [ ]:
#Q05
inference = VariableElimination(model)

evidence={'PeterLate':'True', 'PetersAlarmFails':'False'}
result = inference.query(variables=['JohannaLate'], evidence=evidence)
print(evidence)
print(result)